# Better Way 14.  None을 반환하기보다는 예외를 일으키자

- 파이썬 프로그래머들은 유틸리티 함수를 작성할 때, 반환값 None에 특별한 의미를 부여하는 경향이 있다.
- 예시: 두 숫자를 입력받아 나누는 함수에서 분모에 0이 오는 경우, 결과가 정의되지 않기 때문에 None 반환

In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [2]:
x, y = 5, 0
result = divide(x, y) #ZeroDivisionError에 걸리므로 result = None
if result is None:
    print('Invalid inputs')

Invalid inputs


그러나 문제가 있다. *만약에 분자가 0이 되는 경우 --> 반환값도 0이 된다*
- if문에서 결과를 평가할 때, 0은 (False가 되어) 문제가 될 수 있음
- 함수에서 오류가 발생했는지 알아보기 위해, None 대신 실수로 False에 해당하는 값을 검사할 수 있음

In [3]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs') #return 0인데 None인 경우와 동일하게 작동함

Invalid inputs


In [4]:
'''
위 문제를 해결하는 첫번째 방법: 반환 값을 두 개로 나눠서 튜플에 담음
- 첫번쨰 값은 작업의 성공/실패 여부
- 두번째 값은 계산의 결과
'''
def divide(a, b):
    try:
        return True, a/b
    except ZeroDivisionError:
        return False, None

In [5]:
success, result = divide(x,y)
if not success:
    print('Invalid inputs')

In [6]:
_, result = divide(x,y) #그러나 호출자가 튜플의 첫번째 부분을 쉽게 무시할 수 있음
if not result:
    print('Invalid inputs')

Invalid inputs


In [7]:
'''
두번째 방법: None을 절대로 반환하지 않고, 호출하는 쪽에 예외를 일으켜서 그 예외를 처리하게 함 (옳소!)
'''
def divide(a,b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e #입력값이 잘못됐음을 알리려고 ValueError로 변경함

In [8]:
x, y = 5, 2
try:
    result = divide(x,y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


In [9]:
x, y = 5, 0
try:
    result = divide(x,y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Invalid inputs


In [10]:
x, y = 0, 5
try:
    result = divide(x,y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 0.0


핵심 정리
- 특별한 의미를 나타내려고 None을 반환하는 함수가 오류를 일으키기 쉬운 이유는 None이나 다른 값(예컨데 0, 빈 문자열)이 조건식에서 False로 평가되기 때문이다.
- 특별한 상황을 알릴 때 None을 반환하는 대신에 예외를 일으키자. 문서화가 되어 있다면 호출하는 코드에서 예외를 적절하게 처리할 것이라고 기대할 수 있다.

# Better Way 15. 클로저가 변수 스코프와 상호 작용하는 방법을 알자

문제: 숫자 리스트를 정렬하는데, 특정 그룹에 속한 숫자들이 먼저 오도록 하고 싶음.
- 리스트의 sort 메서드에 헬퍼 함수를 key 인수로 넘겨서 해결 가능
- 이때, 헬퍼 함수의 반환값이 리스트에 있는 각 아이템을 정렬하는 값으로 사용됨

In [11]:
'''
헬퍼는 주어진 아이템이 중요한 그룹에 있는지 확인하고 그에 따라 정렬 키를 다르게 반환한다
'''
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x) #숫자 x가 group에 속하면, 0과 x를 튜플로 반환
        return (1, x)
    values.sort(key=helper)

In [12]:
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}
sort_priority(numbers, group)
print(numbers) #예상한대로 group에 속한 숫자부터 먼저 정렬됨

[2, 3, 5, 7, 1, 4, 6, 8]


- 파이썬의 클로저(closure): 자신이 정의된 스코프에 있는 변수를 참조하는 함수. helper 함수가 group 인수에 접근할 수 있는 이유임
- 파이썬의 함수는 일급 객체 (first-class object)임. 이는 함수를 직접 참조하고, 변수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if 문 등에서 비교할 수 있다는 의미임. 따라서 sort 메서드에서 클로저 함수(=helper)를 key 인수로 받을 수 있음
- 튜플 간의 비교 규칙: 인덱스 0부터 아이템을 비교하고 다음 인덱스 (1, 2, ...) 비교 진행함.
- 클로저 보충 설명: https://whatisthenext.tistory.com/112

In [13]:
'''
새로운 문제: 이번에는 우선순위가 높은 (특정 그룹에 속한) 아이템을 발견했는지 여부를 반환해보자!
'''
def sort_priority2(numbers, group):
    found = False #발견 여부를 저장하는 bool 변수
    def helper(x):
        if x in group:
            found = True #찾았으니까 found 값을 변경
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [14]:
found = sort_priority2(numbers, group)
print('Found:', found) #그러나 이번에는 원하는대로 작동하지 않는다!
print(numbers) #다만 리스트 정렬은 기대한대로 되어있음 (즉, helper 함수가 특정 그룹 아이템을 찾는 동작은 제대로 수행했다는 뜻)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


문제의 이유: **표현식에서 변수를 참조할 때** 파이썬 인터프리터는 참조를 해결하려고 다음 순서로 스코프(scope, 유효 범위)를 탐색함
1. 현재 함수의 스코프
2. (현재 스코프를 담고 있는 다른 함수 같은) 감싸고 있는 스코프
3. 코드를 포함하고 있는 모듈의 스코프 (=전역 스코프)
4. 내장 스코프

이 중 어느 스코프에도 참조한 이름으로 된 변수가 정의되어 있지 않으면 NameError 예외가 발생함!

그러나, (두번째 문제에서처럼) **변수에 값을 할당할 때**는 다른 방식으로 동작한다.
1. 변수가 이미 현재 스코프에 정의되어 있다면 새로운 값을 얻는다. (즉, 값이 업데이트 된다)
2. 변수가 현재 스코프에 존재하지 않으면 변수 정의로 취급한다. 새로 정의하는 변수의 스코프는 그 할당을 포함하고 있는 함수가 된다.

즉, found 변수는 helper 클로저에서 True로 할당되지만, 이는 *sort_priority2에서 일어나는 할당이 아니라 helper 안에서 일어나는 새 변수 정의로 처리되었기 때문에* 잘못 작동한 것이다.

In [15]:
'''
키워드 nonlocal: 특정 변수 이름에 할당할 때, 스코프 탐색이 일어나야 함을 나타냄.
(그러나 위의 탐색 범위에서 2번까지만 진행한다. 즉 전역 스코프 포함, 그 아래까지는 탐색하지 않는다.
'''
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [16]:
found = sort_priority3(numbers, group)
print('Found:', found) 
print(numbers) #이번에는 제대로 작동함

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


그러나 nonlocal은 조심해서 사용해야 한다
- 간단한 함수에서 사용하는 것이 아니라면, 그 부작용을 알아내기 매우 어렵다 (특히 nonlocal 문과 관련 변수에 대한 할당이 멀리 떨어진 긴 함수)

In [17]:
'''
헬퍼 클래스로 상태를 감싸는 방법도 있음. (nonlocal을 사용하기엔 복잡한 경우)
'''
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x): #Better Way 23: 인터페이스가 간단하면 클래스 대신 함수를 받자 -- 참고
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

In [18]:
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True #문제없이 넘어감

파이썬2의 스코프
- 불행하게도 nonlocal 지원하지 않음
- 파이썬의 스코프 규칙을 이용한 다른 방법으로 우회함: 수정가능한(mutable) 리스트/딕셔너리/세트/사용자 정의 클래스의 인스턴스 등 사용

In [ ]:
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True #found[0]의 현재값을 알아내려고 스코프 탐색을 진행하게 됨
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

핵심정리
- 클로저 함수는 자신이 정의된 스코프 중 어디에 있는 변수도 참조할 수 있다.
- 기본적으로 클로저에서 변수를 할당하면 바깥쪽 스코프에는 영향을 미치지 않는다.
- 파이썬3에서는 nonlocal 문을 사용하여 클로저를 감싸고 있는 스코프의 변수를 수정할 수 있음을 알린다.

# Better Way 16. 리스트를 반환하는 대신 제너레이터를 고려하자

In [20]:
'''
문제: 주어진 문자열에 있는 모든 단어의 인덱스 (즉, 첫글자의 인덱스)를 출력
'''
def index_words(text):
    result = [] #결과로 반환할 리스트 선언 및 초기화
    if text:
        result.append(0) #빈 문자열이 아니라면 단어가 1개 이상이 있으므로 인덱스 0 저장
    for index, letter in enumerate(text): #문자열의 모든 문자를 순회
        if letter == ' ':
            result.append(index+1) #공백을 찾을 때마다 (단어와 단어는 공백으로 구분될 테니) 다음 인덱스를 저장
    return result

In [21]:
address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:3]) #세번째 단어까지의 인덱스만 출력

[0, 5, 11]


더 좋은 방법: 제너레이터(generator)를 사용하자!
- 제너레이터는 yield 표현식을 사용
- 제너레이터는 호출 시에 실제로 실행되지 않고 이터레이터(iterator)를 반환
- 내장 함수 next를 호출할 때마다, 이터레이터는 제너레이터가 다음 yield 표현식으로 진행
- 제너레이터가 yield에 전달한 값을 이터레이터가 호출하는 쪽에 반환

In [22]:
'''
다음과 같이 제너레이터를 사용하니,
불필요한 리스트 생성이나 append 메서드 호출, 그로 인한 index+1 등 중요한 부분이 잘 부각되지 않는 문제가 해결됨
'''
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

In [23]:
result = list(index_words_iter(address)) #이터레이터를 리스트로 변환하면 동일한 결과를 쉽게 받을 수 있음.
print(result[:3])

[0, 5, 11]


또한, index_words 함수는 반환하기 전에 모든 결과를 리스트에 저장해야 한다는 문제가 있음
- 입력이 매우 길다면, 프로그램 실행 중에 메모리가 고갈되어 동작이 멈출 것임
- 한편 제너레이터는 (next가 호출될 때마다 yield 포현식으로 넘어가는 방식이라) 다양한 길이의 입력에도 쉽게 이용할 수 있음

In [25]:
'''
파일에서 입력을 한 번에 한 줄씩 읽어서 한 번에 한 단어씩 출력을 내어주는 제너레이터
즉, 입력 한 줄의 최대 길이까지만 메모리를 사용함
'''
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

In [29]:
'''
islice(iterable 객체, [시작], 정지[,step]): 객체를 특정 범위로 슬라이싱하고 iterator로 반환
참고: https://suwoni-codelab.com/python%20%EA%B8%B0%EB%B3%B8/2018/03/07/Python-Basic-itertools/
'''
from itertools import islice

with open('address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 3)
    print(list(results))

[0, 5, 11]


In [35]:
'''
제너레이터 사용 시의 주의점: 반환되는 이터레이터에 상태가 있고 재사용할 수 없다
(Better Way 17. 인수를 순회할 때는 방어적으로 하자)
'''
with open('address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 50) #정지 인덱스로 적당히 큰 숫자 넣어봄
    print(list(results))
    results = islice(it, 50)
    print(list(results)) #위에서 it가 끝까지 도달하고 나니 더이상 반환할 값이 없음 (계속 마지막에 머물러 있음)

[0, 5, 11, 15, 21, 27, 34, 46, 52, 59, 61, 64, 72, 81, 86, 88, 91, 100]
[]


핵심 정리
- 제너레이터를 사용하는 방법이 누적된 결과의 리스트를 반환하는 방법보다 이해하기가 명확하다.
- 제너레이터에서 반환된 이터레이터는 제너레이터 함수의 본문에 있는 yield 표현식에 전달된 값들의 집합이다.
- 제너레이터는 모든 입력과 출력을 메모리에 저장하지 않으므로 입력값의 양을 알기 어려울 때도 연속된 출력을 만들 수 있다.